# ETL DataSUS

## Instalação e configuração das bibliotecas

Referências:
- biblioteca pyreaddbc: https://pypi.org/project/pyreaddbc/
- biblioteca dbfread: https://pypi.org/project/dbfread/
- repositório pyreaddbc issues: https://github.com/osl-incubator/pyreaddbc/issues/19 

In [146]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 KB 10.3 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 12.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install --upgrade --force-reinstall cffi

  Using cached cffi-1.16.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cffi-1.16.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (464 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.22
    Uninstalling pycparser-2.22:
      Successfully uninstalled pycparser-2.22
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


In [2]:
#pip install cffi==1.16.0

In [3]:
pip install pyreaddbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install dbfread

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from platform import python_version
from selenium import webdriver
import sys
import cffi
import pyreaddbc
from dbfread import DBF
import pandas as pd
import itertools

In [6]:
print(python_version())

3.10.12


In [7]:
sys.executable

'/usr/bin/python3.10'

In [8]:
print(cffi.__file__)

/home/nandaoc/.local/lib/python3.10/site-packages/cffi/__init__.py


In [9]:
import cffi
print(cffi.__version__)

1.16.0


## Scraper - extraindo dados da plataforma

In [3]:
import selenium
selenium.__version__

'4.21.0'

## Criando DataFrame

In [132]:
lista_linhas = []

In [108]:
def cria_dbf(caminho_dbc, caminho_dbf):
    # Converte dbc em dbf e salva em um diretório - não te retorno
    arquivo_dbf = pyreaddbc.dbc2dbf(caminho_dbc, caminho_dbf)
    # Converte o arquivo dbf em um objeto iterável
    tabela_dbf = DBF(caminho_dbf)

    return tabela_dbf

In [133]:
def cria_linhas_dados(tabela_dbf):
    i = 0
    iter_tabela_dbf = iter(tabela_dbf)
    iter_tbdbf1, iter_tbdbf2 = itertools.tee(iter_tabela_dbf)

    for itens in iter_tbdbf2:
        linha_dataframe = {}
        for item in itens:
            linha_dataframe[item] = itens[item]
        lista_linhas.append(linha_dataframe)
       
        i+=1

In [134]:
def cria_dataframe(dados):
    df_leitos = pd.DataFrame(dados)

    return df_leitos

In [135]:
tb_dbf = cria_dbf("/home/nandaoc/Downloads/leitos/LTAC2301.dbc", "/home/nandaoc/Downloads/leitos/LTAC2301.dbf")

In [136]:
cria_linhas_dados(tb_dbf)

In [138]:
df = cria_dataframe(lista_linhas)
df.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
0,5701929,120001,001,,,,E,3,00000000000000,3,...,,,5,45,2,0,2,0,202301,1023
1,5701929,120001,001,,,,E,3,00000000000000,3,...,,,2,33,3,0,3,0,202301,1023
2,5701929,120001,001,,,,E,3,00000000000000,3,...,,,4,43,1,0,1,0,202301,1023
3,5701929,120001,001,,,,E,3,00000000000000,3,...,,,6,47,1,0,1,0,202301,1023
4,2001020,120005,002,,,,E,3,04034526001034,3,...,,,4,43,2,0,2,0,202301,1023


## Persistindo dados